In [ ]:
! pip install transformers

In [ ]:
import numpy as np
import pandas as pd
import torch
import transformers

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, train_test_split

In [ ]:
df = pd.read_csv('./train.csv', names=['label', 'review'])

In [ ]:
df.head()

In [ ]:
df = df.iloc[:2000,:]

In [ ]:
model_class, tokenizer_class, pretrained_weights = (transformers.DistilBertModel, transformers.DistilBertTokenizer, 'distilbert-base-uncased')
tokenizer = tokenizer_class.from_pretrained(pretrained_weights)
model = model_class.from_pretrained(pretrained_weights)

In [ ]:
tokenize = df[0].apply(lambda x: tokenizer.encode(x, add_special_tokens=True))

In [ ]:
max_len = 0

for i in tokenize.values:
  if len(i) > max_len:
    max_len = len(i)
  
padded = np.array([i + [0] * (max_len - len(i)) for i in tokenize.values])

In [ ]:
attention_mask = np.where(padded != 0, 1, 0)

In [ ]:
attention_mask[0]

In [ ]:
input_ids = torch.tensor(padded)
attention_mask = torch.tensor(attention_mask)

with torch.no_grad():
  last_hidden_states = model(input_ids, attention_mask=attention_mask)

In [ ]:
features = last_hidden_states[0][:,0,:].numpy()

In [ ]:
labels = df[1]

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, labels, random_state=42)

In [ ]:
lr = LogisticRegression(C=5.2, max_iter = 1000)
lr.fit(x_train, y_train)

LogisticRegression(C=5.2, max_iter=1000)

In [ ]:
lr.score(x_test, y_test)

0.832